In [1]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# load embedding 
vectorstore = FAISS.load_local("../faiss-index",OllamaEmbeddings(model ="llama2:7b-chat-q4_K_M"),allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [2]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [3]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [4]:
from langchain_community.llms.ollama import Ollama

rag_llama = Ollama(model ="llama2:7b-chat-q4_K_M")
lora_llama = Ollama(model = "chih/llama-2-chat-attack")

In [5]:
rag_llama.invoke('hello')
# lora_llama.invoke('hello')

", my dear friend! I'm so glad to hear from you.\nI hope you are doing well and enjoying your day. It's been a while since we last spoke, and I must say, I've missed you terribly! *giggles* I know, I know, it's not like we haven't kept in touch, but you know what I mean! 😉\nAnyway, enough about that. How are things going in your neck of the woods? Have you been up to anything exciting lately? *curious face* Let me tell you all about my day... *excitedly*\nOh wait, I almost forgot! *giggles* I have a surprise for you! 🎉 You're going to love it, I promise! 😘 It's something I've been working on for ages, and I can't wait for you to see it. *excitedly*\nSo, what do you say? Are you ready to catch up and share some fun times together? 🤩 Let me know when you are, my dear! ❤️\nUntil then... stay fabulous, my friend! 💖"

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join( doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | rag_llama
    | StrOutputParser()
)

In [33]:
questions ="""Explain how the technique "Obfuscated Files or Information" operates and its application in avoiding detection.
What techniques might attackers use to discover network services and connections on the target system? Explain the operation principles and potential impact of these techniques.
Describe how attackers utilize application layer protocols (such as web protocols, file transfer protocols, mail protocols, DNS, etc.) for command and control (C2) communications, and discuss possible detection and prevention methods.
What are some of the ways adversaries use proxies to obfuscate their communication in a compromised network?"""

questions = questions.split("\n")
print(len(questions))

11


In [6]:
for j,i in enumerate(questions):
    format = ''
    rag_ans = rag_chain.invoke(i)
    lora_ans = lora_llama.invoke(i)
    format+= f"## {str(j+37)}. {i}\n\n### Rag:\n{rag_ans}\n\n### Lora:\n{lora_ans}\n\n"
    print(format)
    with open("result.md",'a') as f:
        f.write(format)
        f.close()

NameError: name 'questions' is not defined